# Querying Agents

## Imports

In [ ]:
# General
import os
from dotenv import load_dotenv

# Get API keys from .env file
load_dotenv()


# AutoGen
import autogen

# LangChain
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI
from langchain.agents import tool
from datetime import date

## AutoGen

### Define helper methods

In [ ]:
# Define openai configurations
openai_config_list = [
    {
        'model': 'gpt-3.5-turbo',                 # $0.03 for running the first request below (can't get live data, used placeholder data)
        # 'model': 'gpt-4',                           # $0.33 for running the two requests below
        'api_key': os.environ['OPENAI_API_KEY'],
    }
]

### Define the Agent

In [ ]:
# Create AssistantAgent named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "seed": 42,                             # Seed for caching/reproducibility
        "config_list": openai_config_list,             # OpenAI API configurations
        "temperature": 0,                       # Randomness, 0 means no randomness
    }                                           # Config for autogen's enhanced inference API
)

# Create UserProxyAgent named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",                   # NEVER, ALWAYS, or TERMINATE (Recommendation; 1 question: NEVER, multiple followup questions: TERMINATE)
    max_consecutive_auto_reply=10,              # Max number of auto replies
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),   # Termination condition
    code_execution_config={
        "work_dir": "coding",                   # Working directory for code execution (where generated files are stored)
        "use_docker": False,                    # Set to True or image name like "python:3" to use docker
    }
)

### Test the Agent

#### Sum all numbers in a range

In [ ]:
# Assistant receives message from user_proxy
user_proxy.initiate_chat(
    assistant,
    message="""Sum all the numbers from 0 to 10.""",
)

#### Sort array in descending order

In [ ]:
# Assistant receives message from user_proxy
user_proxy.initiate_chat(
    assistant,
    message="""Sort the array [1, 3, 2, 4, 5, 7] in descending order.""",
)

#### Describe Djikstra's algorithm

In [ ]:
# Assistant receives message from user_proxy
user_proxy.initiate_chat(
    assistant,
    message="""Describe Djikstra's algorithm using at most 200 words.""",
)

#### Stock price comparison

In [ ]:
# Assistant receives message from user_proxy
user_proxy.initiate_chat(
    assistant,
    message="""What date is today? Compare the year-to-date gain for META and TESLA.""",
)

Followup question:

In [ ]:
user_proxy.send(
    recipient=assistant,
    message="""Plot a chart of their stock price change YTD and save to stock_price_ytd.png.""",
)

## LangChain

### Define helper methods

In [ ]:
# Get the model
llm_model = "gpt-3.5-turbo"
llm = ChatOpenAI(temperature=0, model=llm_model)

# Create a custom tool
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

# Give the model access to llm-math and wikipedia
tools = load_tools(["llm-math","wikipedia"], llm=llm)

# Give the model access to the custom tool
tools = tools + [time]

### Define the Agent

In [ ]:
# Create an agent with access to llm_math, wikipedia and a custom tool
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

# Create a Python agent
python_agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    handle_parsing_errors=True,
    verbose=True
)

### Test the Agent

#### Sum all numbers in a range

In [ ]:
# agent("""Sum all the numbers from 0 to 10.""")                  # GPT-3.5-turbo: Wrong answer + Confused by output format, endless loop
python_agent("""Sum all the numbers from 0 to 10.""")           # GPT-3.5-turbo: Correct answer + Very fast

#### Sort array in descending order

In [ ]:
# agent("""Sort the array [1, 3, 2, 4, 5, 7] in descending order.""")             # GPT-3.5-turbo: Unable to complete the request (Correct answer) + Confused by output format, endless loop
python_agent("""Sort the array [1, 3, 2, 4, 5, 7] in descending order.""")      # GPT-3.5-turbo: Correct answer + Pretty fast

#### Describe Djikstra's algorithm

In [ ]:
agent("""Describe Djikstra's algorithm using at most 200 words.""")             # GPT-3.5-turbo: Unable to complete the request + Confused by output format, endless loop
# python_agent("""Describe Djikstra's algorithm using at most 200 words.""")      # GPT-3.5-turbo: Unable to complete the request + Confused by output format, crashes

#### Stock price comparison

In [ ]:
# agent("""What date is today? Compare the year-to-date gain for META and TESLA.""")          # GPT-3.5-turbo: Unable to complete the request + Uses Wikipedia to look for stock prices (other than that, correct approach)
python_agent("""What date is today? Compare the year-to-date gain for META and TESLA.""")   # GPT-3.5-turbo: Unable to complete the request + Can't import `datetime`, endless loop

Followup question (must be provided in initial question):

In [ ]:
# agent("""What date is today? Compare the year-to-date gain for META and TESLA. 
#       Plot a chart of their stock price change YTD and save to stock_price_ytd.png.""")          # GPT-3.5-turbo: Unable to complete the request + Uses Wikipedia to look for stock prices (other than that, correct approach)

python_agent("""What date is today? Compare the year-to-date gain for META and TESLA. 
             Plot a chart of their stock price change YTD and save to stock_price_ytd.png.""")   # GPT-3.5-turbo: Unable to complete the request + Makes correct imports, but doesn't look for stock prices
                                                                                                 # GPT-3.5-turbo: Correct answer + Pretty fast (With handle_parse_errors=True)